In [1]:
# Import libraries
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import re
from varname import nameof

import requests
from bs4 import BeautifulSoup
import html
import lxml

In [2]:
def num_cleaning(x):
    try:
        return re.match(r'[\d]*[\.\d]*', x)[0]
    except:
        return x

def to_float(x):
    try:
        return float(x)
    except:
        return x

def mapper(data):
    try:
        data.shape[1]       # This is actually to check whether it is a DataFrame or not
        return data.applymap(num_cleaning).applymap(to_float)
    except:
        return data.map(num_cleaning).map(to_float)

def gram_to_liter(x):
    return x * 0.001

def iu_to_mcg(x):
    # This is from international units to mcg retinol
    # I'll use this function to convert vimain a units to something that I can compare with the australian recommendation for daily intake
    return x * 0.3

def key_nutrients():
    return ["protein", "water", "fiber", "vitamin_a", "thiamin", "riboflavin", "niacin", "vitamin_b6", "vitamin_b12", "folate", "vitamin_c", "calcium", "irom", "magnesium", "potassium", "sodium", "zink"]



In [3]:
def nutrition_prep(filename):
    nutrition = pd.read_csv(filename)
    nutrition.set_index("name", inplace = True)

    filter_ = key_nutrients()

    nutrition = nutrition[filter_]

    nutrition = mapper(nutrition)

    nutrition["water"] = nutrition["water"].map(gram_to_liter)
    nutrition["vitamin_a"] = nutrition["vitamin_a"].map(iu_to_mcg)

    return nutrition

def food_selector(foodname, df):
    return df.loc[foodname]

In [4]:
def dailyintake_info(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "lxml")

    di_table = soup.find(id = "tbl-calc")
    di_rows = di_table.find_all("tr")

    di_dict = {}

    for row in di_rows:
        items = row.find_all("td")
        if len(items) > 1:
            di_dict[items[0].text] = items[1].text

    s = pd.Series(di_dict)

    return s

def dailyintake_prep(serie):
    serie = mapper(serie)
    serie.drop("Iodine", inplace = True)
    serie.name = "daily_intake"
    serie.index = key_nutrients()
    return serie

In [5]:
def foodquality(food, dailyintake):
    if len(food) == len(dailyintake):
        s = (food / dailyintake) * 100
        s = s.sort_values(ascending = False)
        s = s.reset_index()
        s.columns = ["nutrient", "%OfDailyIntake"]
        return s

In [30]:
foodname1 = "Cauliflower, raw"
foodname2 = "Cornstarch"
filename = "../data/Nutritional_values.csv"
url_w_30 = "https://www.eatforhealth.gov.au/node/1813927/done?sid=806757&token=05ce5572f5618ac641c9f2395b28c59f"

In [31]:
df = nutrition_prep(filename)
food1 = food_selector(foodname1, df)
food2 = food_selector(foodname2, df)

In [15]:
di = dailyintake_info(url_w_30)
di = dailyintake_prep(di)
pd.DataFrame(di).T

,protein,water,fiber,vitamin_a,thiamin,riboflavin,niacin,vitamin_b6,vitamin_b12,folate,vitamin_c,calcium,irom,magnesium,potassium,sodium,zink
daily_intake,46.0,2.1,25.0,700.0,1.1,1.1,14.0,1.3,2.4,400.0,45.0,1000.0,18.0,310.0,2800.0,460.0,8.0


In [9]:
def foodquality(food, dailyintake):
    if len(food) == len(dailyintake):
        df = pd.merge(food, dailyintake, how = "outer", left_index = True, right_index = True)
        df["%OfDailyIntake"] = (food / dailyintake) * 100
        return df.T

In [10]:
foodquality(food, di)

,protein,water,fiber,vitamin_a,thiamin,riboflavin,niacin,vitamin_b6,vitamin_b12,folate,vitamin_c,calcium,irom,magnesium,potassium,sodium,zink
"Cauliflower, raw",1.920000,0.092070,2.0,0.0,0.050000,0.060000,0.507000,0.184000,0.0,57.00,48.200000,22.0,0.420000,15.00000,299.000000,30.000000,0.270
daily_intake,46.000000,2.100000,25.0,700.0,1.100000,1.100000,14.000000,1.300000,2.4,400.00,45.000000,1000.0,18.000000,310.00000,2800.000000,460.000000,8.000
%OfDailyIntake,4.173913,4.384286,8.0,0.0,4.545455,5.454545,3.621429,14.153846,0.0,14.25,107.111111,2.2,2.333333,4.83871,10.678571,6.521739,3.375


In [27]:
def foodquality2(dailyintake, foods):
    df = pd.DataFrame(dailyintake)
    count = 1
    for food in foods:
        if len(food) == len(dailyintake):
            df = pd.merge(df, food, how = "outer", left_index = True, right_index = True)
            df["%OfDailyIntake_" + str(count)] = (food / dailyintake) * 100
            count += 1

    return df.T

In [32]:
foodquality2(di, [food1, food2])

,protein,water,fiber,vitamin_a,thiamin,riboflavin,niacin,vitamin_b6,vitamin_b12,folate,vitamin_c,calcium,irom,magnesium,potassium,sodium,zink
daily_intake,46.000000,2.100000,25.0,700.0,1.100000,1.100000,14.000000,1.300000,2.4,400.00,45.000000,1000.0,18.000000,310.000000,2800.000000,460.000000,8.000
"Cauliflower, raw",1.920000,0.092070,2.0,0.0,0.050000,0.060000,0.507000,0.184000,0.0,57.00,48.200000,22.0,0.420000,15.000000,299.000000,30.000000,0.270
%OfDailyIntake_1,4.173913,4.384286,8.0,0.0,4.545455,5.454545,3.621429,14.153846,0.0,14.25,107.111111,2.2,2.333333,4.838710,10.678571,6.521739,3.375
Cornstarch,0.260000,0.008320,0.9,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000,2.0,0.470000,3.000000,3.000000,9.000000,0.060
%OfDailyIntake_2,0.565217,0.396190,3.6,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.2,2.611111,0.967742,0.107143,1.956522,0.750


In [23]:
len(food)

17

In [33]:
df.head(10)

,protein,water,fiber,vitamin_a,thiamin,riboflavin,niacin,vitamin_b6,vitamin_b12,folate,vitamin_c,calcium,irom,magnesium,potassium,sodium,zink
name,,,,,,,,,,,,,,,,,
Cornstarch,0.26,0.00832,0.9,0.0,0.000,0.000,0.000,0.000,0.00,0.0,0.0,2.0,0.47,3.0,3.0,9.0,0.06
"Nuts, pecans",9.17,0.00352,9.6,16.8,0.660,0.130,1.167,0.210,0.00,22.0,1.1,70.0,2.53,121.0,410.0,0.0,4.53
"Eggplant, raw",0.98,0.09230,3.0,6.9,0.039,0.037,0.649,0.084,0.00,22.0,2.2,9.0,0.23,14.0,229.0,2.0,0.16
"Teff, uncooked",13.30,0.00882,8.0,2.7,0.390,0.270,3.363,0.482,0.00,0.0,0.0,180.0,7.63,184.0,427.0,12.0,3.63
"Sherbet, orange",1.10,0.06610,1.3,13.8,0.027,0.097,0.063,0.023,0.13,4.0,2.3,54.0,0.14,8.0,96.0,46.0,0.48
"Cauliflower, raw",1.92,0.09207,2.0,0.0,0.050,0.060,0.507,0.184,0.00,57.0,48.2,22.0,0.42,15.0,299.0,30.0,0.27
"Taro leaves, raw",4.98,0.08566,3.7,1447.5,0.209,0.456,1.513,0.146,0.00,126.0,52.0,107.0,2.25,45.0,648.0,3.0,0.41
"Lamb, raw, ground",16.56,0.05947,0.0,0.0,0.110,0.210,5.960,0.130,2.31,18.0,0.0,16.0,1.55,21.0,222.0,59.0,3.41
"Cheese, camembert",19.80,0.05180,0.0,246.0,0.028,0.488,0.630,0.227,1.30,62.0,0.0,388.0,0.33,20.0,187.0,842.0,2.38
